In [23]:
import os
import pandas as pd


In [24]:
import os
os.chdir('c:\\Users\\Vigneswaran\\OneDrive - Datanetiix Solutions Inc\\Desktop\\Diesease_Prognosis')


In [25]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))


In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    min_samples_split: int
    min_samples_leaf: int
    random_state: int
    target_column: str
    n_components: float  # PCA components as a percentage of variance to retain


In [27]:
from diseaseprognosis.constants import *
from diseaseprognosis.utils.common import read_yaml, create_directories

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest
        params_pca = self.params.PCA
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            min_samples_split = params.min_samples_split,
            random_state = params.random_state,
            min_samples_leaf = params.min_samples_leaf,
            target_column = schema.name,
            n_components=params_pca.n_components  # PCA parameters

        )

        return model_trainer_config


In [29]:
import pandas as pd
import os
from diseaseprognosis import logger
from sklearn.ensemble import RandomForestClassifier
import joblib

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import joblib
import os

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
        # Initialize PCA with the number of components from the config
        self.pca = PCA(n_components=self.config.n_components, random_state=self.config.random_state)

        # Initialize Random Forest model with parameters from the config
        self.model = RandomForestClassifier(
            n_estimators=self.config.n_estimators,
            min_samples_split=self.config.min_samples_split,
            random_state=self.config.random_state
        )

    def train(self):
        # Load training and testing data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Split into features and target
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # Apply PCA to reduce dimensions
        train_x = self.pca.fit_transform(train_x)
        test_x = self.pca.transform(test_x)


        # Train the Random Forest model
        self.model.fit(train_x, train_y)

        # Save the trained Random Forest model to a separate file
        rf_model_path = os.path.join(self.config.root_dir, f"{self.config.model_name}_rf")
        joblib.dump(self.model, rf_model_path)
        print(f"Random Forest model saved at {rf_model_path}")

        # Save the PCA components to a separate file
        pca_model_path = os.path.join(self.config.root_dir, f"{self.config.model_name}_pca")
        joblib.dump(self.pca, pca_model_path)
        print(f"PCA components saved at {pca_model_path}")



        return rf_model_path, pca_model_path


In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    rf_model_path, pca_model_path = model_trainer.train()
except Exception as e:
    raise e


[2024-12-04 11:15:31,866: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-04 11:15:31,870: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-04 11:15:31,886: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-04 11:15:31,889: INFO: common: created directory at: artifacts]
[2024-12-04 11:15:31,892: INFO: common: created directory at: artifacts/model_trainer]
Model saved at artifacts/model_trainer\model.joblib
